In [1]:
import sys

sys.path.append('/home/max/myrepos/vlstm_cuda/')

import torch
torch.set_printoptions(linewidth=200, threshold=100000)
import triton
import triton.language as tl
import math

In [2]:
triton.runtime.driver.active.get_current_target()

GPUTarget(backend='cuda', arch=89, warp_size=32)

In [3]:
from src_triton.mlstm_parallel_fw_v0.qkv import qkv_fw

In [4]:
S = 128 #32 #32 #16 #8 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 32 # dim per head
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [5]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(1) #TODO from here: with seed=0 even the pytorch version alone breaks for float16 and bfloat16
# fixed:
# qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
# qs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# # vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# vs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 100.
# # vs = torch.zeros((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
# vs[:,:,1,0] = 7.
# qs[:,:,1,0] = 1.

# vs[:,:,1,16] = 8.
# qs[:,:,1,16] = 1.
# random: 
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
# igs = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S, 1) / 10.
# igs = torch.zeros((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
igs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
# fgs = torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
fgs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE)

dHs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)

In [6]:
# inputs float16
dtype_fp16 = torch.float16
qs_half = qs.to(dtype=dtype_fp16)
ks_half = ks.to(dtype=dtype_fp16)
vs_half = vs.to(dtype=dtype_fp16)
igs_half = igs.to(dtype=dtype_fp16)
fgs_half = fgs.to(dtype=dtype_fp16)
dHs_half = dHs.to(dtype=dtype_fp16)

In [7]:
# inputs bfloat16
dtype_bf16 = torch.bfloat16
qs_bf16 = qs.to(dtype=dtype_bf16)
ks_bf16 = ks.to(dtype=dtype_bf16)
vs_bf16 = vs.to(dtype=dtype_bf16)
igs_bf16 = igs.to(dtype=dtype_bf16)
fgs_bf16 = fgs.to(dtype=dtype_bf16)
dHs_bf16 = dHs.to(dtype=dtype_bf16)

In [8]:
hs_half, m_half, n_half = qkv_fw(matQ=qs_half, matK=ks_half, matV=vs_half)

Triton grid: (2, 1, 1), BLOCK_Q: 64, BLOCK_KV: 32


In [9]:
qkv_half = ((qs_half @ ks_half.transpose(-2, -1))/math.sqrt(DH)) @ vs_half

In [10]:
hs_half[:, :, :, :10]

tensor([[[[ 5.5117e+00, -6.2500e+00, -2.2910e+00, -7.6875e+00, -1.0102e+01,  1.2688e+01, -8.2266e+00,  2.6312e+01, -1.9703e+01,  2.3484e+01],
          [ 1.6234e+01,  3.3105e+00,  6.9766e+00, -3.9238e+00, -1.7578e+00,  4.4824e-01, -5.5547e+00, -7.8125e+00,  1.0570e+01,  9.8359e+00],
          [ 3.6074e+00,  5.1641e+00,  1.7250e+01,  1.4617e+01,  2.3797e+01,  4.1168e-02,  3.0742e+00, -4.4570e+00, -7.2812e+00, -2.6641e+01],
          [-6.1680e+00, -1.0125e+01, -9.6875e+00, -1.4658e+00,  3.6797e+00, -2.8149e-01, -5.0078e+00,  1.0211e+01, -4.6016e+00, -1.1289e+01],
          [-4.5000e+00,  5.5117e+00,  4.1289e+00, -5.7812e+00, -7.5000e+00,  1.6781e+01,  9.4766e+00, -1.2516e+01, -1.1477e+01,  2.3578e+01],
          [-8.4453e+00,  1.9625e+01, -1.8984e+00, -1.5391e+00, -1.0266e+01,  5.0898e+00, -3.9258e+00, -6.4219e+00, -4.6992e+00,  1.0461e+01],
          [-4.0039e+00, -1.4551e+00,  1.7256e+00,  4.8711e+00,  1.4289e+01,  4.5586e+00,  6.5742e+00,  2.2156e+01,  6.4414e+00, -1.6359e+01],
      

In [11]:
qkv_half[:, :, :, :10]

tensor([[[[ 5.5156e+00, -6.2500e+00, -2.2930e+00, -7.6875e+00, -1.0109e+01,  1.2680e+01, -8.2188e+00,  2.6312e+01, -1.9703e+01,  2.3500e+01],
          [ 1.6219e+01,  3.3086e+00,  6.9688e+00, -3.9297e+00, -1.7539e+00,  4.4922e-01, -5.5547e+00, -7.8125e+00,  1.0570e+01,  9.8359e+00],
          [ 3.5957e+00,  5.1602e+00,  1.7250e+01,  1.4617e+01,  2.3797e+01,  4.6875e-02,  3.0781e+00, -4.4609e+00, -7.2891e+00, -2.6656e+01],
          [-6.1758e+00, -1.0125e+01, -9.6875e+00, -1.4648e+00,  3.6797e+00, -2.7930e-01, -5.0117e+00,  1.0219e+01, -4.6016e+00, -1.1297e+01],
          [-4.5078e+00,  5.5156e+00,  4.1211e+00, -5.7891e+00, -7.5000e+00,  1.6766e+01,  9.4766e+00, -1.2523e+01, -1.1477e+01,  2.3578e+01],
          [-8.4375e+00,  1.9625e+01, -1.8994e+00, -1.5430e+00, -1.0273e+01,  5.0898e+00, -3.9258e+00, -6.4219e+00, -4.6953e+00,  1.0461e+01],
          [-4.0078e+00, -1.4463e+00,  1.7246e+00,  4.8711e+00,  1.4289e+01,  4.5547e+00,  6.5703e+00,  2.2156e+01,  6.4336e+00, -1.6375e+01],
      

In [12]:
res_diff = hs_half - qkv_half

In [13]:
res_diff[:, :, -10:, :10], res_diff.shape

(tensor([[[[-0.0156, -0.0078, -0.0039, -0.0078,  0.0000,  0.0156,  0.0000,  0.0117,  0.0000,  0.0000],
           [-0.0015,  0.0020, -0.0039,  0.0068, -0.0059,  0.0039,  0.0000,  0.0000, -0.0054,  0.0000],
           [-0.0059,  0.0000, -0.0029, -0.0039,  0.0000, -0.0039, -0.0039,  0.0039,  0.0000,  0.0000],
           [ 0.0078,  0.0039, -0.0020,  0.0078, -0.0078,  0.0000, -0.0039,  0.0000,  0.0000,  0.0000],
           [-0.0078,  0.0000, -0.0039, -0.0020,  0.0000,  0.0005,  0.0078, -0.0078, -0.0078,  0.0000],
           [ 0.0000, -0.0039,  0.0000,  0.0000,  0.0078,  0.0078, -0.0049,  0.0000,  0.0039,  0.0000],
           [-0.0049,  0.0000, -0.0039, -0.0156,  0.0010, -0.0020,  0.0000, -0.0039,  0.0000,  0.0000],
           [-0.0078, -0.0020, -0.0156,  0.0000, -0.0009,  0.0000,  0.0020, -0.0029,  0.0156,  0.0000],
           [ 0.0000,  0.0000, -0.0020,  0.0000, -0.0020,  0.0039,  0.0000,  0.0000, -0.0078,  0.0029],
           [ 0.0000,  0.0000, -0.0078,  0.0029,  0.0020,  0.0000,  0.0000

In [17]:
torch.max(torch.abs(res_diff))

tensor(0.0312, device='cuda:0', dtype=torch.float16)

In [14]:
qs_half.stride(0), qs_half.stride(1), qs_half.stride(2), qs_half.stride(3)

(4096, 4096, 32, 1)

In [15]:
3 & 1

1

In [16]:
bool(3 & 2)

True